In [1]:
# Lab 9 XOR
import torch

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

이제 XOR 문제에 해당되는 입력과 출력을 정의합니다.

In [3]:
X = torch.FloatTensor([[0, 0],
                       [0, 1], 
                       [1, 0], 
                       [1, 1]]).to(device)
Y = torch.FloatTensor([[0], 
                       [1], 
                       [1], 
                       [0]]).to(device)

이제 1개의 뉴런을 가지는 단층 퍼셉트론을 구현해봅시다.    
단층 퍼셉트론이 처음 소개되었을 때는 계단 함수였지만,    
우리는 이미 또 다른 활성화 함수인 시그모이드 함수를 알고 있으므로    
시그모이드 함수를 사용해보겠습니다.

In [4]:
# nn layers
linear = torch.nn.Linear(2, 1, bias=True)
sigmoid = torch.nn.Sigmoid()

In [5]:
# model
model = torch.nn.Sequential(linear, sigmoid).to(device)

0 또는 1을 예측하는 이진 분류 문제이므로 비용 함수로는 크로스엔트로피 함수를 사용합니다.   
nn.BCELoss()는 이진 분류에서 사용하는 크로스엔트로피 함수입니다.

In [6]:
# define cost/loss & optimizer
criterion = torch.nn.BCELoss().to(device)        # Binary Cross Entropy 
optimizer = torch.optim.SGD(model.parameters(), lr=1)

In [7]:
for step in range(10001):
    optimizer.zero_grad()
    hypothesis = model(X)

    # cost/loss function
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if step % 1000 == 0:
        print(step, cost.item())

0 0.7273974418640137
1000 0.6931471824645996
2000 0.6931471824645996
3000 0.6931471824645996
4000 0.6931471824645996
5000 0.6931471824645996
6000 0.6931471824645996
7000 0.6931471824645996
8000 0.6931471824645996
9000 0.6931471824645996
10000 0.6931471824645996


학습된 단층 퍼셉트론의 예측값 확인하기

총 10,001회 학습한 단층 퍼셉트론의 예측값도 확인해보겠습니다.

In [9]:
# Accuracy computation
# True if hypothesis>0.5 else False
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), '\nCorrect: ', predicted.detach().cpu().numpy(), '\nAccuracy: ', accuracy.item())


Hypothesis:  [[0.5]
 [0.5]
 [0.5]
 [0.5]] 
Correct:  [[0.]
 [0.]
 [0.]
 [0.]] 
Accuracy:  0.5


실제값은 0, 1, 1, 0 임에도 예측값은 0, 0, 0, 0으로 문제를 풀지 못하는 모습을 보여줍니다.